In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [35]:
import random as ran
from IPython.display import clear_output
from src import spotify, dp, utils, DB

db = DB()

In [36]:
genres_cat = ['k-pop', 'pop', 'hip-hop', 'ballad', 'jazz']

genres_dict = dict({
    "k-pop": "3dbLT62Cvs46Ju7a8gpr36",
    "pop": "3gzIK2dGSVvPBiohsTwEWZ",
    "hip-hop": "2FWquqPNxte8iqZ3ATQG0p",
    "ballad": "3UPjb91Fwm7u2tAm92Bk0p",
    "jazz": "7tIOwGFRTM08JdAeKOELGp"
})

In [46]:
mail_box_seed_list = list()

for _ in range(0, 10):
    genre_cnt = ran.randrange(1, 4)
    genre_cat = list()
    
    while len(genre_cat) != genre_cnt:
        ran_genres_cat = genres_cat[
            ran.randrange(0, len(genres_cat))
        ]
        
        if ran_genres_cat not in genre_cat:
            genre_cat.append(ran_genres_cat)

    mail_box_seed_list.append(genre_cat)
    
mail_box_seed_list

[['pop', 'jazz'],
 ['hip-hop', 'pop', 'k-pop'],
 ['ballad'],
 ['ballad'],
 ['pop', 'ballad', 'hip-hop'],
 ['pop', 'hip-hop', 'ballad'],
 ['pop', 'jazz'],
 ['hip-hop'],
 ['hip-hop', 'jazz'],
 ['k-pop', 'hip-hop', 'ballad']]

In [54]:
# 1. Get Token
token = spotify.get_token()

mail_boxes_list = list()
sel_track_list = list()
sel_tracks = pd.DataFrame()

for box_seed in mail_box_seed_list:
    sel_tracks = pd.DataFrame()
    
    for seed_genre in box_seed:
        seed_id = genres_dict[seed_genre]
        
        sel_tracks = sel_tracks.append(spotify.direct_sel(token, seed_id), ignore_index=True)
    
    sel_track_list.append(sel_tracks)
    mail_box_id = str(db.regist_mail_box(sel_tracks).inserted_id)
    
    mail_boxes_list.append(mail_box_id)

print(mail_boxes_list)
sel_track_list[0]

['623c3b95a96fb402f17e7195', '623c3b96a96fb402f17e7196', '623c3b96a96fb402f17e7197', '623c3b96a96fb402f17e7198', '623c3b96a96fb402f17e7199', '623c3b96a96fb402f17e719a', '623c3b97a96fb402f17e719b', '623c3b97a96fb402f17e719c', '623c3b97a96fb402f17e719d', '623c3b97a96fb402f17e719e']


,id,name,artists,artists_name
0,3gzIK2dGSVvPBiohsTwEWZ,Easy,3WGpXCj9YhhfX11TToZcXP,Troye Sivan
1,7tIOwGFRTM08JdAeKOELGp,Blue room,3rxeQlsv0Sc2nyYaZ5W71T,Chet Baker
